In [268]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
import os
import requests.auth
from rich import print


import reddit_creds as rc

In [47]:
CLIENT_ID = rc.CLIENT_ID
CLIENT_SECRET = rc.CLIENT_SECRET
USERNAME = rc.USERNAME
PASSWORD = rc.PASSWORD

In [194]:
client_auth = requests.auth.HTTPBasicAuth(CLIENT_ID, CLIENT_SECRET)
post_data = {'grant_type': 'password', 'username': USERNAME, 'password':PASSWORD}


In [195]:
headers = {
    'User-Agent': "A red scrapper"
}

In [196]:
#Token Access ID
TOKEN_ACCESS_ENDPOINT = "https://www.reddit.com/api/v1/access_token"
response = requests.post(TOKEN_ACCESS_ENDPOINT,data=post_data,headers=headers, auth = client_auth)
if response.status_code == 200:
    token_id = response.json()['access_token']

In [197]:
OAUTH_ENDPOINT = 'https://oauth.reddit.com'
params_get = {
    'limit' : 100
}
headers_get = {
    'User-Agent': "A red scrapper",
    'Authorization': "Bearer " + token_id,
}

In [199]:
response = requests.get(OAUTH_ENDPOINT + '/r/personalfinance/', headers=headers_get, params = params_get)
print(response)

<Response [200]>

In [280]:
def get_subs(subreddit):
    temp_df = pd.DataFrame()
    i=0
    params_get = {
        'limit' : 100
    }
    for i in range(0,10):
        response = requests.get(OAUTH_ENDPOINT + subreddit, headers=headers_get, params = params_get)
        data = response.json()['data']['children']
        after_key = response.json()['data']['after']
        before_key = response.json()['data']['before']
        for post in data:
            temp_df = temp_df.append({
                'subreddit': post['data']['subreddit'],
                'text': post['data']['selftext']
            }, ignore_index = True)
        params_get = {
            'limit' : 100,
            'after': after_key
        }
        i += 1
    return temp_df

In [298]:
####Tool to check makeup of text in subreddit, subs like sports have little to no textual content

temp = get_subs('/r/poems/')
print('Percent empty strings: ', round((temp['text'].values == '').sum()/len(temp),2)*100, '%')

Percent empty strings:  3.0 %

In [299]:
subs = ['/r/personalfinance/',
        '/r/personaltraining/',
        '/r/poems/',
        '/r/keepwriting/',
        '/r/story/']

In [300]:
df = pd.DataFrame()

In [301]:
df_list = []
for sub in subs:
    temp_df = get_subs(sub)
    df_list.append(temp_df)
df = pd.concat(df_list,axis=0)

In [314]:
print('There are ', len(df), ' records.')
print('Percent empty strings: ', (round((df['text'].values == '').sum()/len(df),2))*100, '%')


There are  4773  records.

Percent empty strings:  7.000000000000001 %